In [ ]:
MutationFile = 'Cutaneous_somatic_mutation_MELA_AU_DNVs_sorted_dinuc_filter.bed'
FastaFile = 'hg19.fa'
OutputFile = 'AC>TT_Mutation_Context_Frequency.csv'

from SequenceContext import CreateNewBedFile
CreateNewBedFile(1,1,MutationFile,FastaFile)

from CompareMutationContext import AttainMutationData
AttainMutationData('AC>TT', OutputFile)



In [ ]:
MutationFile = 'Cutaneous_somatic_mutation_MELA_AU_DNVs_sorted_dinuc_filter.bed'
OutputFile = 'AC>TT_Mutation_Context_Frequency.csv'
FastaFile = 'hg19.fa'
from ActuallyNormalizeData import NormalizeData
NormalizeData(OutputFile, FastaFile, 4)


In [ ]:
# Bar graphs
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
OutputFile = 'AC>TT_Mutation_Context_Frequency.csv'
gdata = pd.read_csv('Normalized_'+OutputFile)
gdata = gdata.sort_values('normalizedCounts', ascending = False)
sns.set_theme(style = 'whitegrid')
sns.barplot(y ='sequenceContext', x = 'normalizedCounts', data = gdata,)

In [ ]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
OutputFile = 'AC>TT_Mutation_Context_Frequency.csv'
with open('Linear_Regression_'+OutputFile, 'r') as data:
    df = pd.read_csv(data)
    countDict = {}
    xdf = df.drop(columns=['context','frequency'])
    ydf = df.frequency
    for column in xdf:
        rValue, _ = scipy.stats.pearsonr(xdf[column], df.frequency)
        countDict[column] = rValue
    i = 0
    fivePrimeDict = {}
    threePrimeDict = {}
    for key, value in countDict.items():
        if i < 4:
            fivePrimeDict[key] = value
            i += 1
        else:
            threePrimeDict[key] = value
    threeDrop = (min(threePrimeDict, key = lambda dictionaryKey: abs(threePrimeDict.get(dictionaryKey))))
    fiveDrop = (min(fivePrimeDict, key = lambda dictionaryKey: abs(fivePrimeDict.get(dictionaryKey))))
    Adi = df.drop(columns=['context', 'frequency', 'threePrimeA', 'threePrimeT', 'threePrimeC', 'threePrimeG', str(fiveDrop)])
    Ezra = df.drop(columns=['context', 'frequency', 'fivePrimeA', 'fivePrimeT', 'fivePrimeC', 'fivePrimeG', str(threeDrop)])
    X2 = sm.add_constant(Adi)
    est = sm.OLS(ydf,X2).fit()
    print(est.summary())
    X3 = sm.add_constant(Ezra)
    est = sm.OLS(ydf,X3).fit()
    print(est.summary())